# Notes on rank-constrained output-error system identification

Let's load some packages:
- *numpy* for matrix multiplication
- *matplotlib* for visualizing results
- *cvxpy* for convex optimization

In [2]:
import numpy as np
import matplotlib.pyplot as plt


## Introduction to system identification
To introduce the relevand concepts for system identification, consider the simple example of a mass-spring-damper system

![mass-spring-damper](https://upload.wikimedia.org/wikipedia/commons/thumb/4/45/Mass_spring_damper.svg/1200px-Mass_spring_damper.svg.png)

affected by an exogenous force $F$. Newton's second loaw of motion gives the second-order differential equation

$$ m\ddot{x} = -k\dot{x} -cx + F $$.


### State-space representation
The mass-spring-damper system can be expressed on a state-space form

$$
\begin{bmatrix}
\dot{x}\\
\ddot{x}
\end{bmatrix}
=
\begin{bmatrix}
 0 &1\\-c/m &-k/m
\end{bmatrix}
\begin{bmatrix}
x\\
\dot{x}
\end{bmatrix}
+
\begin{bmatrix}
0\\1/m
\end{bmatrix}
F
$$

Assume we are given measurements $y(k)$ of the position $x$ at discrete time instance $t = kT_\mathrm{s}$, $K=0,1,...$, with a constant time $\Delta T$. The system can be rewritten as a discrete-time, linear, time invariante state-space model of the form (e.g. using [zero-order hold](https://en.wikipedia.org/wiki/Discretization))

$$
\begin{align}
 x(k+1) &= Ax(k) + Bu(k)\\
 y(k) &= Cx(k)
\end{align}
$$

where the state vector $x(k)in\mathbb{R}^{n}$ consost of the position and velocity at time instance $k\Delta T$ and the input $u(k) = F(k\Delta T) \in\mathbb{R}^{m}$. Matrices $A\in\mathbb{R}^{n\times n}$, $B\in\mathbb{R}^{n\times m}$, $C\in\mathbb{R}^{l\times n}$. Repeated substitution of the state equation in the the measurement equation.

$$
\begin{bmatrix}
y(0)\\
y(1)\\
y(2)\\
\vdots
\end{bmatrix}
=
\begin{bmatrix}
C\\
CA\\
CA^2\\
\vdots
\end{bmatrix}
x(0)
+
\begin{bmatrix}
0 &0 &0 &\cdots\\
CB &0 &0 &\cdots\\
CAB &CB &0 &\cdots\\
\vdots &\ddots &\ddots &\ddots\\
\end{bmatrix}
\begin{bmatrix}
u(0)\\
u(1)\\
u(2)\\
\vdots
\end{bmatrix}
$$

For $x(0)=0$, the system can be representad as

$$
y(k) = \sum_{l=1}^\infty  CA^{l-1}B u(k-l) = \sum_{l=1}^{\infty}  h(l) u(k-l),
$$

which is called the impulse-response representation of the system since, passing an impulse through the system gives the impulse response

$$
h(k) = CA^{k-1}B.
$$

Since, in system identification, the impulse response $h$ is unknown, it is often coinvenient to rewrite the system in the following form

$$
\begin{bmatrix}
y(1)\\
y(2)\\
y(3)\\
y(4)\\
\vdots
\end{bmatrix}
=
\begin{bmatrix}
u(0) &0     &0    &0    &\cdots\\
u(1) &u(0) &0    &0    &\cdots\\
u(2) &u(1) &u(0) &0    &\cdots\\
u(3) &u(2) &u(1) &u(0) &\ddots\\
\vdots &\ddots &\ddots &\ddots &\ddots
\end{bmatrix}
\begin{bmatrix}
h(1)\\h(2)\\h(3)\\h(4)\\ \vdots
\end{bmatrix}.
$$
compactly we use the matrix notation

$$
y = Uh
$$

### Output-error system identification

The output-error system identification can be summarized as follows:

**Problem 1 (OE)** Given $N$ input-output measurements $y(1),y(2),...,y(N)$ and $u(0),u(1),...,u(N-1)$, identify the first $N$ impulse response coefficients $h(1),h(2),...,h(N)$ and find an $r$th order realization $(A,B,C)$.

*Remark*: The (partial) realization problem of finding the system matrices $(A,B,C)$ from a finite set of impulse-resoonse coefficients $h(1),h(2),...,h(N-1)$, practically means finding and infinite and extension of the impulse response

To formulate the OE problem, we make use of the following key results

**Lemma 1** A system $G$ of order $r$, denoted $\mathrm{deg}(G) = r$, satisfies
$$ r = \mathrm{deg}(H) = \mathrm{rank}(H)$$
where $H$ is the Hankel operator
$$
H =
\begin{bmatrix}
h(1) &h(2)  &\cdots\\
h(2) &h(3)  &\cdots\\
\vdots &\vdots 
\end{bmatrix}.
$$

Furthermore, given a truncated Hankel matrix

$$
H_{N/2,N/2} = 
\begin{bmatrix}
     h(1) &h(2) &\cdots &h(N/2)\\
     h(2) &h(3) &\cdots &h(N/2+1)\\
     \vdots &\vdots & &\vdots\\
     h(N/2) &h(N/2+1) &\cdots &h(N)
\end{bmatrix}
$$

with $\mathrm{rank}(H_{N/2,N/2}) = r < N/2$, it is possible to find a minimal system realization $(A,B,C)$ of order $r$, with the $N$ first impulse-reponse coefficients satisfying

$$
h(k) = CA^{k-1}B,~\text{for}~k=1,2,...,N.
$$

using the [Ho-Kalman algorithm](https://www.degruyter.com/document/doi/10.1524/auto.1966.14.112.545/html).

Thus, we formulate the rank-constrained system identification problem

$$
\begin{align}
&\underset{
h=\{h_0,h_1,...,h_N\} }{\text{minimize}} ~\|y-Uh\|_2^2 \\ 
&\text{subject to}\\
&\mathrm{rank} \left( H_{N/2,N/2} \right) \leq r.
\end{align}
$$

This formulation is as close to 'exact' as can be, with the only approximation errors due to the truncation of the impulse response, which for stable systems can ba made small by selecting $N$ sufficiently large. Denoting the true system $G$ and the estimated one $\hat{G}$, errors caused by expanging the impulse response are bounded by (<span style="color:red">verify this</span>)

$$
\| G - \hat{G} \|_{\mathrm{op}} \leq \sum_{k=N+1}^\infty ( CA^{k-1}B )^2.
$$


## Example
Let's generate some noisy data

In [4]:
# PRBS noise input signal (not the most exact implementation, but it will do for now)
N = 10
u = np.round(2*(np.random.rand(N,1)-0.5))
#plt.plot(u)

